In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit import QuantumCircuit
from qiskit.primitives import Estimator
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)

# Load and preprocess the Wine dataset (binary classification)
wine = load_wine()
X, y = wine.data, wine.target
X, y = X[y != 2], y[y != 2]

# Standardize and reduce to 2D
X_scaled = StandardScaler().fit_transform(X)
X_pca = PCA(n_components=2).fit_transform(X_scaled)
y_encoded = LabelEncoder().fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_pca, y_encoded, test_size=0.2, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Quantum circuit setup
num_qubits = 2
feature_map = ZZFeatureMap(feature_dimension=num_qubits)
ansatz = TwoLocal(num_qubits, rotation_blocks='ry', entanglement_blocks='cz', reps=3)

qc = QuantumCircuit(num_qubits)
qc.compose(feature_map, inplace=True)
qc.compose(ansatz, inplace=True)

print("\n🔍 Quantum Circuit:")
print(qc.draw(fold=100))

# Estimator and QNN
estimator = Estimator(options={"shots": 1024})
qnn = EstimatorQNN(
    circuit=qc,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    estimator=estimator
)
qnn_layer = TorchConnector(qnn)

# PyTorch Model
class QuantumClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.classical_pre = nn.Linear(2, 2)
        self.qnn = qnn_layer
        self.fc1 = nn.Linear(1, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.out = nn.Linear(4, 1)

    def forward(self, x):
        x = self.classical_pre(x)
        x = self.qnn(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        return self.out(x)  # No sigmoid here

model = QuantumClassifier()

# Training setup
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()
epochs = 100

print("\n🚀 Training Started...")
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    
    # Optional: Gradient clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

    optimizer.step()

    with torch.no_grad():
        pred_probs = torch.sigmoid(outputs)
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {loss.item():.4f} | Output Range: {pred_probs.min().item():.4f} to {pred_probs.max().item():.4f}")

# Evaluation
model.eval()
with torch.no_grad():
    test_logits = model(X_test_tensor)
    test_probs = torch.sigmoid(test_logits)
    predictions = (test_probs > 0.5).float()
    accuracy = (predictions == y_test_tensor).float().mean()
    print(f"\n✅ Test Accuracy: {accuracy:.4f}")



🔍 Quantum Circuit:
     ┌──────────────────────────┐┌────────────────────────────────────────────────────┐
q_0: ┤0                         ├┤0                                                   ├
     │  ZZFeatureMap(x[0],x[1]) ││  TwoLocal(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5],θ[6],θ[7]) │
q_1: ┤1                         ├┤1                                                   ├
     └──────────────────────────┘└────────────────────────────────────────────────────┘

🚀 Training Started...


C:\Users\Aravind\AppData\Local\Temp\ipykernel_10236\3711114133.py:50: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator(options={"shots": 1024})
C:\Users\Aravind\AppData\Local\Temp\ipykernel_10236\3711114133.py:51: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


Epoch 1/100 - Loss: 0.7210 | Output Range: 0.4244 to 0.4280
Epoch 2/100 - Loss: 0.7183 | Output Range: 0.4312 to 0.4328
Epoch 3/100 - Loss: 0.7162 | Output Range: 0.4354 to 0.4369
Epoch 4/100 - Loss: 0.7142 | Output Range: 0.4393 to 0.4411
Epoch 5/100 - Loss: 0.7122 | Output Range: 0.4433 to 0.4456
Epoch 6/100 - Loss: 0.7103 | Output Range: 0.4472 to 0.4502
Epoch 7/100 - Loss: 0.7084 | Output Range: 0.4512 to 0.4547
Epoch 8/100 - Loss: 0.7067 | Output Range: 0.4552 to 0.4591
Epoch 9/100 - Loss: 0.7050 | Output Range: 0.4593 to 0.4631
Epoch 10/100 - Loss: 0.7038 | Output Range: 0.4632 to 0.4655
Epoch 11/100 - Loss: 0.7027 | Output Range: 0.4669 to 0.4679
Epoch 12/100 - Loss: 0.7018 | Output Range: 0.4702 to 0.4702
Epoch 13/100 - Loss: 0.7010 | Output Range: 0.4726 to 0.4726
Epoch 14/100 - Loss: 0.7002 | Output Range: 0.4749 to 0.4749
Epoch 15/100 - Loss: 0.6994 | Output Range: 0.4773 to 0.4773
Epoch 16/100 - Loss: 0.6987 | Output Range: 0.4796 to 0.4796
Epoch 17/100 - Loss: 0.6980 | Out